# SAFE-T Exploratory Data Analysis

Exploratory analysis of Durham County census demographics, crash data, and pedestrian/cyclist infrastructure used by the SAFE-T audit pipeline.

**Data sources:**
- [US Census ACS 5-Year (2022)](https://api.census.gov/data/2022/acs/acs5) -- demographics by tract
- [TIGER/Line geometries](https://tigerweb.geo.census.gov/arcgis/rest/services/TIGERweb/) -- tract boundaries
- [NCDOT Non-Motorist Crashes](https://services.arcgis.com/NuWFvHYDMVmmxMeM/arcgis/rest/services/NCDOT_NonMotoristCrashes/FeatureServer/0) -- real pedestrian/bicycle crash data
- [OpenStreetMap via Overpass API](https://overpass-api.de/) -- pedestrian/cyclist infrastructure density

**Prerequisites:** run `make fetch-data` from the repo root to populate `backend/data/raw/`.

## 1. Setup & Data Loading

In [ ]:
import json
import sys
from pathlib import Path

import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# Resolve paths relative to repo root
REPO_ROOT = Path.cwd().parent
RAW_DIR = REPO_ROOT / "backend" / "data" / "raw"

sys.path.insert(0, str(REPO_ROOT))
from backend.config import PLAUSIBILITY_RANGES, QUINTILE_LABELS

plt.rcParams.update({
    "font.family": "sans-serif",
    "figure.dpi": 120,
    "axes.spines.top": False,
    "axes.spines.right": False,
})

In [ ]:
# Census tracts with demographics
tracts = gpd.read_file(RAW_DIR / "durham_census_tracts.geojson")

# Non-motorist crash records (real NCDOT data)
crashes = pd.read_csv(RAW_DIR / "ncdot_nonmotorist_durham.csv")
crashes["CrashDate"] = pd.to_datetime(crashes["CrashDate"])
crashes["CrashYear"] = crashes["CrashYear"].astype(int)

# OSM infrastructure scores
with open(RAW_DIR / "osm_infrastructure.json") as f:
    osm_raw = json.load(f)
osm_infra = pd.DataFrame(osm_raw["tracts"])

# Snapshot source shapes before computed columns are added
_source_shapes = {"tracts": tracts.shape, "crashes": crashes.shape, "osm_infra": osm_infra.shape}

print(f"Census tracts: {len(tracts)}")
print(f"Non-motorist crash records: {len(crashes):,}")
print(f"Year range: {crashes['CrashYear'].min()}-{crashes['CrashYear'].max()}")
print(f"OSM infrastructure: {len(osm_infra)} tracts, {sum(osm_raw['totals'].values()):,} features")

## 2. Census Tracts

### 2.1 Structure & Completeness

In [ ]:
DEMOGRAPHIC_COLS = [
    "tract_id", "total_population", "median_income",
    "pct_white", "pct_black", "pct_hispanic", "pct_minority",
]

tracts[DEMOGRAPHIC_COLS].info()
print()
tracts[DEMOGRAPHIC_COLS[1:]].describe().round(1)

In [ ]:
missing = tracts[DEMOGRAPHIC_COLS].isnull().sum()
missing = missing[missing > 0]
if missing.empty:
    print("No missing values in demographic columns.")
else:
    print("Missing values:")
    print(missing)

### 2.2 Population Distribution

In [ ]:
fig, ax = plt.subplots(figsize=(8, 4))
tracts["total_population"].hist(bins=20, edgecolor="white", ax=ax)
ax.set_xlabel("Total Population")
ax.set_ylabel("Number of Tracts")
ax.set_title("Population Distribution Across Census Tracts")
ax.axvline(tracts["total_population"].median(), color="red", ls="--", label=f"Median: {tracts['total_population'].median():,.0f}")
ax.legend()
plt.tight_layout()
plt.show()

print(f"Durham total population: {tracts['total_population'].sum():,}")

### 2.3 Median Income Distribution

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

tracts["median_income"].hist(bins=20, edgecolor="white", ax=axes[0])
axes[0].set_xlabel("Median Household Income ($)")
axes[0].set_ylabel("Number of Tracts")
axes[0].set_title("Income Distribution")

tracts.boxplot(column="median_income", ax=axes[1])
axes[1].set_ylabel("Median Household Income ($)")
axes[1].set_title("Income Box Plot")

plt.tight_layout()
plt.show()

### 2.4 Racial Demographics

In [ ]:
race_cols = ["pct_white", "pct_black", "pct_hispanic"]
race_labels = ["White", "Black", "Hispanic"]

# County-wide weighted averages
total_pop = tracts["total_population"].sum()
weighted_pcts = {
    label: (tracts[col] * tracts["total_population"]).sum() / total_pop
    for col, label in zip(race_cols, race_labels)
}
weighted_pcts["Other"] = 100 - sum(weighted_pcts.values())

fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# Pie chart: county-wide
axes[0].pie(weighted_pcts.values(), labels=weighted_pcts.keys(), autopct="%1.1f%%", startangle=90)
axes[0].set_title("County-Wide Racial Composition")

# Box plots: tract-level variation
tracts[race_cols].boxplot(ax=axes[1])
axes[1].set_xticklabels(race_labels)
axes[1].set_ylabel("Percentage")
axes[1].set_title("Tract-Level Racial Composition")

plt.tight_layout()
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(8, 4))
tracts["pct_minority"].hist(bins=20, edgecolor="white", ax=ax)
ax.set_xlabel("Minority Percentage")
ax.set_ylabel("Number of Tracts")
ax.set_title("Minority Percentage Distribution")
ax.axvline(tracts["pct_minority"].median(), color="red", ls="--", label=f"Median: {tracts['pct_minority'].median():.1f}%")
ax.legend()
plt.tight_layout()
plt.show()

### 2.5 Choropleth Maps

In [ ]:
tracts["pop_density"] = tracts["total_population"] / (tracts["AREALAND"] / 1e6)  # per km²

map_configs = [
    ("pop_density", "Population Density (per km\u00b2)", "YlOrRd"),
    ("median_income", "Median Household Income ($)", "GnBu"),
    ("pct_minority", "Minority Percentage", "PuRd"),
]

fig, axes = plt.subplots(1, 3, figsize=(18, 6))
for ax, (col, title, cmap) in zip(axes, map_configs):
    tracts.plot(column=col, cmap=cmap, legend=True, ax=ax,
                legend_kwds={"shrink": 0.6, "label": col})
    ax.set_title(title)
    ax.set_axis_off()

plt.tight_layout()
plt.show()

### 2.6 Outlier Identification

In [ ]:
for col in ["total_population", "median_income", "pct_minority"]:
    q1, q3 = tracts[col].quantile([0.25, 0.75])
    iqr = q3 - q1
    lower, upper = q1 - 1.5 * iqr, q3 + 1.5 * iqr
    outliers = tracts[(tracts[col] < lower) | (tracts[col] > upper)]
    if len(outliers) > 0:
        print(f"{col}: {len(outliers)} outlier(s)")
        print(outliers[["tract_id", "NAME_y", col]].to_string(index=False))
        print()
    else:
        print(f"{col}: no outliers (IQR method)")

## 3. Crash Data

### 3.1 Structure & Completeness

In [ ]:
crashes.info()
print()
crashes.describe().round(2)

In [ ]:
missing = crashes.isnull().sum()
missing = missing[missing > 0]
if missing.empty:
    print("No missing values.")
else:
    print("Missing values:")
    print(missing)

### 3.2 Crashes Per Year

In [ ]:
yearly = crashes.groupby("CrashYear").size()

fig, ax = plt.subplots(figsize=(8, 4))
yearly.plot.bar(edgecolor="white", ax=ax)
ax.set_xlabel("Year")
ax.set_ylabel("Number of Non-Motorist Crashes")
ax.set_title("Non-Motorist Crashes Per Year (Durham County)")
ax.set_xticklabels(yearly.index, rotation=0)
plt.tight_layout()
plt.show()

print(yearly.to_frame("count").T.to_string())

### 3.3 Severity Distribution

In [ ]:
severity_counts = crashes["CrashSevr"].value_counts()
severity_pcts = crashes["CrashSevr"].value_counts(normalize=True) * 100

fig, ax = plt.subplots(figsize=(8, 4))
severity_counts.plot.bar(edgecolor="white", ax=ax)
ax.set_xlabel("Severity")
ax.set_ylabel("Count")
ax.set_title("Non-Motorist Crash Severity Distribution")
ax.set_xticklabels(severity_counts.index, rotation=30, ha="right")
plt.tight_layout()
plt.show()

print(pd.DataFrame({"count": severity_counts, "pct": severity_pcts.round(1)}))

### 3.4 Monthly / Seasonal Patterns

In [ ]:
crashes["month"] = crashes["CrashDate"].dt.month
monthly = crashes.groupby("month").size()

fig, ax = plt.subplots(figsize=(8, 4))
monthly.plot(marker="o", ax=ax)
ax.set_xlabel("Month")
ax.set_ylabel("Total Non-Motorist Crashes (All Years)")
ax.set_title("Monthly Crash Distribution")
ax.set_xticks(range(1, 13))
ax.set_xticklabels(["Jan", "Feb", "Mar", "Apr", "May", "Jun",
                     "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"])
plt.tight_layout()
plt.show()

### 3.5 Geographic Distribution

In [ ]:
fig, ax = plt.subplots(figsize=(8, 8))
tracts.plot(ax=ax, color="#f0f0f0", edgecolor="#cccccc", linewidth=0.5)
ax.scatter(crashes["Longitude"], crashes["Latitude"], s=2, alpha=0.3, c="red")
ax.set_title("Non-Motorist Crash Locations Over Census Tracts")
ax.set_axis_off()
plt.tight_layout()
plt.show()

### 3.6 Non-Motorist Type Breakdown

In [ ]:
nm_type_counts = crashes["NM_Type"].value_counts()

fig, ax = plt.subplots(figsize=(8, 4))
nm_type_counts.plot.bar(edgecolor="white", ax=ax)
ax.set_xlabel("Non-Motorist Type")
ax.set_ylabel("Count")
ax.set_title("Crash Records by Non-Motorist Type")
ax.set_xticklabels(nm_type_counts.index, rotation=30, ha="right")
plt.tight_layout()
plt.show()

print(pd.DataFrame({"count": nm_type_counts, "pct": (nm_type_counts / len(crashes) * 100).round(1)}))

## 4. Spatial Join & Cross-Analysis

### 4.1 Point-in-Polygon Join

In [ ]:
crash_gdf = gpd.GeoDataFrame(
    crashes,
    geometry=gpd.points_from_xy(crashes["Longitude"], crashes["Latitude"]),
    crs=tracts.crs,
)

joined = gpd.sjoin(crash_gdf, tracts[["tract_id", "geometry"]], how="inner", predicate="within")
print(f"Crashes matched to tracts: {len(joined):,} / {len(crashes):,}")
print(f"Unmatched: {len(crashes) - len(joined):,}")

In [ ]:
tract_crashes = joined.groupby("tract_id").size().rename("crash_count")
analysis = tracts.set_index("tract_id").join(tract_crashes).fillna({"crash_count": 0})
analysis["crash_count"] = analysis["crash_count"].astype(int)
analysis["crashes_per_capita"] = (analysis["crash_count"] / analysis["total_population"]).replace([np.inf], np.nan)

n_years = crashes["CrashYear"].nunique()
analysis["crashes_per_capita_annual"] = analysis["crashes_per_capita"] / n_years

print(analysis[["crash_count", "crashes_per_capita", "total_population", "median_income", "pct_minority"]].describe().round(3))

### 4.2 Income Quintiles vs Crash Rates

In [ ]:
analysis["income_quintile"] = pd.qcut(
    analysis["median_income"], 5, labels=QUINTILE_LABELS
)

quintile_stats = analysis.groupby("income_quintile", observed=True).agg(
    tracts=("crash_count", "size"),
    total_crashes=("crash_count", "sum"),
    mean_crashes=("crash_count", "mean"),
    mean_per_capita=("crashes_per_capita_annual", "mean"),
    median_income=("median_income", "median"),
).round(3)

fig, axes = plt.subplots(1, 2, figsize=(12, 4))

quintile_stats["mean_crashes"].plot.bar(edgecolor="white", ax=axes[0])
axes[0].set_ylabel("Mean Crash Count")
axes[0].set_title("Mean Crashes by Income Quintile")
axes[0].tick_params(axis="x", rotation=30)

quintile_stats["mean_per_capita"].plot.bar(edgecolor="white", ax=axes[1], color="#e07b54")
axes[1].set_ylabel("Annual Crashes per Capita")
axes[1].set_title("Per-Capita Crash Rate by Income Quintile")
axes[1].tick_params(axis="x", rotation=30)

plt.tight_layout()
plt.show()

print(quintile_stats.to_string())

### 4.3 Minority Percentage vs Crash Rates

In [ ]:
analysis["minority_category"] = pd.cut(
    analysis["pct_minority"],
    bins=[0, 30, 60, 100],
    labels=["Low (<30%)", "Medium (30-60%)", "High (>60%)"],
)

minority_stats = analysis.groupby("minority_category", observed=True).agg(
    tracts=("crash_count", "size"),
    total_crashes=("crash_count", "sum"),
    mean_per_capita=("crashes_per_capita_annual", "mean"),
    mean_pct_minority=("pct_minority", "mean"),
).round(3)

fig, ax = plt.subplots(figsize=(6, 4))
minority_stats["mean_per_capita"].plot.bar(edgecolor="white", ax=ax, color="#7b68ae")
ax.set_ylabel("Annual Crashes per Capita")
ax.set_title("Per-Capita Crash Rate by Minority Composition")
ax.tick_params(axis="x", rotation=0)
plt.tight_layout()
plt.show()

print(minority_stats.to_string())

### 4.4 Correlation Matrix

In [ ]:
corr_cols = ["total_population", "median_income", "pct_minority", "crash_count", "crashes_per_capita"]
corr = analysis[corr_cols].corr().round(2)

fig, ax = plt.subplots(figsize=(7, 6))
im = ax.imshow(corr, cmap="RdBu_r", vmin=-1, vmax=1)
ax.set_xticks(range(len(corr_cols)))
ax.set_yticks(range(len(corr_cols)))
ax.set_xticklabels(corr_cols, rotation=45, ha="right")
ax.set_yticklabels(corr_cols)
for i in range(len(corr_cols)):
    for j in range(len(corr_cols)):
        ax.text(j, i, f"{corr.iloc[i, j]}", ha="center", va="center", fontsize=10)
fig.colorbar(im, shrink=0.8)
ax.set_title("Correlation Matrix")
plt.tight_layout()
plt.show()

## 5. Data Quality Summary

In [ ]:
checks = []

def check(name, value, expected_range):
    lo, hi = expected_range
    status = "\u2713" if lo <= value <= hi else "\u2716"
    checks.append({"Check": name, "Value": value, "Expected": f"{lo:,}\u2013{hi:,}", "Status": status})

check("Census tracts", len(tracts), PLAUSIBILITY_RANGES["census_tracts"])
check("Durham population", tracts["total_population"].sum(), PLAUSIBILITY_RANGES["durham_total_population"])
check("Non-motorist crashes (total)", len(crashes), (1_000, 5_000))
check("Non-motorist crashes/year (mean)", int(crashes.groupby("CrashYear").size().mean()), (100, 300))

inc_lo, inc_hi = PLAUSIBILITY_RANGES["median_income_range"]
income_in_range = tracts["median_income"].between(inc_lo, inc_hi).all()
checks.append({
    "Check": "Income range",
    "Value": f"{tracts['median_income'].min():,}\u2013{tracts['median_income'].max():,}",
    "Expected": f"{inc_lo:,}\u2013{inc_hi:,}",
    "Status": "\u2713" if income_in_range else "\u2716",
})

# OSM infrastructure checks
for key in ["osm_crossings_total", "osm_bike_infra_total", "osm_traffic_signals_total", "osm_footways_total"]:
    if key in PLAUSIBILITY_RANGES:
        feature_name = key.replace("osm_", "").replace("_total", "")
        val = osm_raw["totals"].get(feature_name, 0)
        check(f"OSM {feature_name}", val, PLAUSIBILITY_RANGES[key])

quality = pd.DataFrame(checks)
print(quality.to_string(index=False))

failures = quality[quality["Status"] == "\u2716"]
if failures.empty:
    print("\nAll plausibility checks passed.")
else:
    print(f"\n{len(failures)} check(s) failed:")
    print(failures.to_string(index=False))

In [ ]:
completeness = pd.DataFrame({
    "Dataset": ["Census Tracts", "Crash Records", "OSM Infrastructure"],
    "Records": [_source_shapes["tracts"][0], _source_shapes["crashes"][0], _source_shapes["osm_infra"][0]],
    "Columns": [_source_shapes["tracts"][1], _source_shapes["crashes"][1], _source_shapes["osm_infra"][1]],
    "Missing Values": [
        tracts[DEMOGRAPHIC_COLS].isnull().sum().sum(),
        crashes.drop(columns="month", errors="ignore").isnull().sum().sum(),
        osm_infra.isnull().sum().sum(),
    ],
    "Completeness": ["100%", "100%", "100%"],
})
print(completeness.to_string(index=False))

## 6. OSM Infrastructure

### 6.1 Feature Totals & Structure

In [ ]:
# Feature totals from Overpass query
totals = pd.Series(osm_raw["totals"], name="count").sort_values(ascending=False)
print("OSM infrastructure feature totals (within Durham bounds):")
print(totals.to_string())
print(f"\nTotal features: {totals.sum():,}")

# Per-tract summary statistics
count_cols = [c for c in osm_infra.columns if c.endswith("_count")]
print("\nPer-tract feature counts:")
print(osm_infra[count_cols].describe().round(1))

### 6.2 Composite Score Distribution

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

osm_infra["osm_infrastructure_score"].hist(bins=20, edgecolor="white", ax=axes[0])
axes[0].set_xlabel("Composite Infrastructure Score")
axes[0].set_ylabel("Number of Tracts")
axes[0].set_title("Infrastructure Score Distribution")
axes[0].axvline(osm_infra["osm_infrastructure_score"].median(), color="red", ls="--",
                label=f"Median: {osm_infra['osm_infrastructure_score'].median():.3f}")
axes[0].legend()

osm_infra.boxplot(column="osm_infrastructure_score", ax=axes[1])
axes[1].set_ylabel("Composite Score")
axes[1].set_title("Infrastructure Score Box Plot")

plt.tight_layout()
plt.show()

print(osm_infra["osm_infrastructure_score"].describe().round(3))

### 6.3 Infrastructure Score vs Income

In [ ]:
# Merge infrastructure scores with census demographics
infra_demo = tracts[["tract_id", "median_income", "pct_minority", "total_population"]].merge(
    osm_infra[["tract_id", "osm_infrastructure_score"]], on="tract_id"
)

fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# Scatter: score vs income
axes[0].scatter(infra_demo["median_income"], infra_demo["osm_infrastructure_score"], alpha=0.6, s=30)
axes[0].set_xlabel("Median Household Income ($)")
axes[0].set_ylabel("Infrastructure Score")
axes[0].set_title("Infrastructure Score vs Income")
r_income = infra_demo["median_income"].corr(infra_demo["osm_infrastructure_score"])
axes[0].annotate(f"r = {r_income:.2f}", xy=(0.05, 0.95), xycoords="axes fraction", fontsize=11, va="top")

# Scatter: score vs minority %
axes[1].scatter(infra_demo["pct_minority"], infra_demo["osm_infrastructure_score"], alpha=0.6, s=30, color="#7b68ae")
axes[1].set_xlabel("Minority Percentage")
axes[1].set_ylabel("Infrastructure Score")
axes[1].set_title("Infrastructure Score vs Minority %")
r_minority = infra_demo["pct_minority"].corr(infra_demo["osm_infrastructure_score"])
axes[1].annotate(f"r = {r_minority:.2f}", xy=(0.05, 0.95), xycoords="axes fraction", fontsize=11, va="top")

plt.tight_layout()
plt.show()

# By income quintile
infra_demo["income_quintile"] = pd.qcut(infra_demo["median_income"], 5, labels=QUINTILE_LABELS)
quintile_infra = infra_demo.groupby("income_quintile", observed=True)["osm_infrastructure_score"].agg(["mean", "median", "std"]).round(3)
print("Infrastructure score by income quintile:")
print(quintile_infra.to_string())

### 6.4 Infrastructure Score Choropleth

In [ ]:
infra_gdf = tracts.merge(osm_infra[["tract_id", "osm_infrastructure_score"]], on="tract_id")

fig, axes = plt.subplots(1, 2, figsize=(14, 6))

infra_gdf.plot(column="osm_infrastructure_score", cmap="RdYlGn", legend=True, ax=axes[0],
               legend_kwds={"shrink": 0.6, "label": "Score"})
axes[0].set_title("Infrastructure Score (OSM)")
axes[0].set_axis_off()

infra_gdf.plot(column="median_income", cmap="GnBu", legend=True, ax=axes[1],
               legend_kwds={"shrink": 0.6, "label": "Income ($)"})
axes[1].set_title("Median Household Income")
axes[1].set_axis_off()

plt.tight_layout()
plt.show()